In [ ]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import mean_squared_error

In [ ]:
plt.rcParams["figure.figsize"] = [16, 9]
plt.rcParams["figure.dpi"] = 300
plt.rcParams["font.size"] = 20
plt.rcParams["axes.labelsize"] = 20
plt.rcParams["axes.titlesize"] = 24
plt.rcParams["xtick.labelsize"] = 16
plt.rcParams["ytick.labelsize"] = 16
plt.rcParams["font.family"] = "serif"

In [ ]:
DATA = "../data/"
!ls $DATA/

In [ ]:
df = pd.read_csv(f"{DATA}/ohlc_DowJones/ohlc_DowJones_val.csv")
df = df.filter(regex="mid_price")
df.columns

In [ ]:
plt.rcParams["font.size"] = 10
corr = df.corr(numeric_only=True)
fig, ax = plt.subplots(figsize=(20, 15))
sb.heatmap(corr, cmap="Blues", annot=True, ax=ax)

In [ ]:
PATH_PICKLE_PRICE_DOWJONES = (
    "../storage/thesis-gan/abllixzn/preds_epoch=241-target_price=mid_price-target_volume=None.pickle"
)
with open(PATH_PICKLE_PRICE_DOWJONES, "rb") as handle:
    pred_prices_dowjones_dict = pickle.load(handle)
pred_prices_dowjones_dict.keys()

In [ ]:
df_preds = pd.DataFrame(pred_prices_dowjones_dict["pred_prices"].T)
df_preds.columns = df.columns

In [ ]:
corr_preds = df_preds.corr(numeric_only=True)
fig, ax = plt.subplots(figsize=(20, 15))
sb.heatmap(corr_preds, cmap="Blues", annot=True, ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 15))
sb.heatmap((corr - corr_preds) / corr, cmap="Blues", annot=True, ax=ax)

In [ ]:
mse = mean_squared_error(corr["Value"], corr_preds["Value"])
mse
